# GPT for Tiny Shakespere Dataset
following Andrej Karpathy's Neural Network Course

In [17]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-11-08 15:19:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-11-08 15:19:36 (9.81 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [43]:
with open("input.txt", 'r',  encoding='utf-8') as file:
    text = file.read()

In [44]:
all_chars = sorted(set(text))
print("".join(all_chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


all_chars has all the characters in the entire dataset for the model to train in

In [45]:
vocab_size = len(all_chars)
vocab_size

65

In [54]:
stoi = {ch:i for i, ch in enumerate(all_chars)}
itos = {i:ch for i, ch in enumerate(all_chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [59]:
decode(encode("Hi"))

'Hi'

In [63]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [60]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [62]:
len(train_data), len(val_data)

(1003854, 111540)